# The aim of this notebook is to clean the provided locations from the collected CSV files (via open data Toronto), retrieve their geolocations (via openstreetmaps and google api) and merge bus & streetcar data together into one file. 

In [1]:
import pandas as pd
import glob
import googlemaps
from datetime import datetime
#from geopy.geocoders import Nominatim
from geopy.geocoders import Bing
from tqdm import tqdm
from geopy.extra.rate_limiter import RateLimiter
import time

## import and combine streetcar data

In [2]:
glob.glob('*.xlsx')

['ttc-streetcar-delay-data-2014.xlsx',
 'ttc-streetcar-delay-data-2015.xlsx',
 'ttc-streetcar-delay-data-2016.xlsx',
 'ttc-streetcar-delay-data-2017.xlsx',
 'ttc-streetcar-delay-data-2018.xlsx',
 'ttc-streetcar-delay-data-2019.xlsx',
 'ttc-streetcar-delay-data-2020.xlsx',
 'ttc-streetcar-delay-data-readme.xlsx']

In [3]:
df = pd.concat(pd.read_excel('ttc-streetcar-delay-data-2014.xlsx', sheet_name=None), ignore_index=True)
df1 = pd.concat(pd.read_excel('ttc-streetcar-delay-data-2015.xlsx', sheet_name=None), ignore_index=True)
df2 = pd.concat(pd.read_excel('ttc-streetcar-delay-data-2016.xlsx', sheet_name=None), ignore_index=True)
df3 = pd.concat(pd.read_excel('ttc-streetcar-delay-data-2017.xlsx', sheet_name=None), ignore_index=True)
df4 = pd.concat(pd.read_excel('ttc-streetcar-delay-data-2018.xlsx', sheet_name=None), ignore_index=True)
df5 = pd.concat(pd.read_excel('ttc-streetcar-delay-data-2019.xlsx', sheet_name=None), ignore_index=True)
df6 = pd.concat(pd.read_excel('ttc-streetcar-delay-data-2020.xlsx', sheet_name=None), ignore_index=True)

In [29]:
df_total = pd.concat([df, df1, df2, df3, df4, df5, df6])

df_total.shape

(82702, 13)

In [30]:
df_total.to_csv('StreetcarDelayData.csv')

In [31]:
df_total.head()

,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,Incident ID,Delay,Gap
0,2014-01-02,505,06:31:00,Thursday,Dundas and Roncesvalles,Late Leaving Garage,4.0,8.0,E/B,4018.0,NaN,NaN,NaN
1,2014-01-02,504,12:43:00,Thursday,King and Shaw,Utilized Off Route,20.0,22.0,E/B,4128.0,NaN,NaN,NaN
2,2014-01-02,501,14:01:00,Thursday,Kingston road and Bingham,Held By,13.0,19.0,W/B,4016.0,NaN,NaN,NaN
3,2014-01-02,504,14:22:00,Thursday,King St. and Roncesvalles Ave.,Investigation,7.0,11.0,W/B,4175.0,NaN,NaN,NaN
4,2014-01-02,504,16:42:00,Thursday,King and Bathurst,Utilized Off Route,3.0,6.0,E/B,4080.0,NaN,NaN,NaN


In [37]:
df_total['Min Delay'] = df_total['Min Delay'].fillna(0)
df_total['Delay'] = df_total['Delay'].fillna(0)

In [40]:
df_total['Min Gap'] = df_total['Min Delay'].fillna(0)
df_total['Gap'] = df_total['Delay'].fillna(0)

In [38]:
df_total['Min Delay2'] = df_total['Min Delay'] + df_total['Delay']

In [41]:
df_total['Min Gap2'] = df_total['Min Gap'] + df_total['Gap']

In [44]:
df_total['Min Delay'] = df_total['Min Delay2']
df_total['Min Gap'] = df_total['Min Gap2']

In [50]:
df_total = df_total.drop(columns = ['Delay', 'Gap', 'Min Delay2', 'Min Gap2', 'Incident ID'])

In [51]:
df_total.to_csv('StreetcarDelayData.csv')

In [52]:
df_total

,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle
0,2014-01-02,505,06:31:00,Thursday,Dundas and Roncesvalles,Late Leaving Garage,4.0,4.0,E/B,4018.0
1,2014-01-02,504,12:43:00,Thursday,King and Shaw,Utilized Off Route,20.0,20.0,E/B,4128.0
2,2014-01-02,501,14:01:00,Thursday,Kingston road and Bingham,Held By,13.0,13.0,W/B,4016.0
3,2014-01-02,504,14:22:00,Thursday,King St. and Roncesvalles Ave.,Investigation,7.0,7.0,W/B,4175.0
4,2014-01-02,504,16:42:00,Thursday,King and Bathurst,Utilized Off Route,3.0,3.0,E/B,4080.0
...,...,...,...,...,...,...,...,...,...,...
4172,2020-07-26,501,12:58:00,Sunday,QUEEN AND DUFFERIN,Mechanical,10.0,10.0,W/B,4422.0
4173,2020-07-27,301,02:09:00,Monday,QUEEN AND YORK,Mechanical,25.0,25.0,E/B,4514.0
4174,2020-07-27,306,02:20:00,Monday,Dundas West Station,Emergency Services,19.0,19.0,E/B,3129.0
4175,2020-08-03,504,00:56:00,Monday,Dufferin Loop,Mechanical,10.0,10.0,E/B,4464.0


In [57]:
location = df_total['Location'].tolist()

In [58]:
location

['Dundas and Roncesvalles',
 'King and Shaw',
 'Kingston road and Bingham',
 'King St. and Roncesvalles Ave.',
 'King and Bathurst',
 'Queen and Beaconsfeild',
 'Roncesvalles and King Street West',
 'Spadina and St. Andrews',
 'Broadview and Queen',
 'Bathurst and St. Clair',
 'Queen and Roncesvalles',
 'King at Bathurst',
 'Queen and Roncesvalles',
 'Roncesvalles and Queen',
 'Bathurst at St. Clair',
 'Bathurst at St Clair',
 'Roncesvalles and Queen',
 'Roncevalles and Howard Park',
 'Bathurst and St Clair',
 'Roncesvalles and Howard Park',
 'King and Bathurst',
 'King at Bathurst',
 'Bathurst and St. Clair',
 'King at Bathurst',
 'King at Bathurst',
 'King at Bathurst',
 'Roncesvalles and Howard Park',
 'Queen and Roncesvalles',
 'Dundas West Station',
 'Roncesvalles and Howard Park',
 'King at Bathurst',
 'Queen and Roncesvalles',
 'Roncesvalles and Queen',
 'Queen St. E. and Rainsford',
 'Roncesvalles and Howard Park',
 'Roncesvalles and Queen',
 'Roncesvalles and Howard Park',
 'Q

In [63]:
len(location)

82702

# Import bus data and combine datasets
# Find long, lat coordinates 

In [166]:
df_bus = pd.read_csv('BusDelayData_updated.csv')
df_st = pd.read_csv('StreetcarDelayData.csv')

In [167]:
print(df_bus.shape)
print(df_st.shape)

(479403, 11)
(82702, 11)


In [168]:
df_bus.describe()

,Unnamed: 0,Route,Min Delay,Min Gap,Vehicle
count,479403.000000,479403.000000,455863.000000,460895.000000,409515.000000
mean,37124.607172,120.442763,19.312943,29.327068,5601.028744
std,23136.356327,233.267134,71.212563,72.377669,3296.670030
min,0.000000,1.000000,-54.000000,-2.000000,0.000000
25%,17121.000000,38.000000,6.000000,12.000000,1602.000000
50%,35786.000000,72.000000,10.000000,20.000000,7654.000000
75%,55761.000000,116.000000,15.000000,30.000000,8317.000000
max,94216.000000,106117.000000,6518.000000,6528.000000,86175.000000


### create column for categorizing bus and streetcar prior to merging datasets

In [169]:
df_bus['vtype'] = 'bus'

df_bus

,Unnamed: 0,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,vtype
0,40771,2016-07-24,506,5:40:00,Sunday,Howardpark and Dundas,Diversion,6518.0,6528.0,B/W,7403.0,bus
1,18711,2015-03-19,32,5:00:00,Thursday,Eglinton and Richardson,Diversion,5775.0,5783.0,b/w,NaN,bus
2,15286,2014-02-19,73,18:22:00,Wednesday,ALBION & ISLINGTON,Diversion,3780.0,398.0,S,8044.0,bus
3,87495,2014-11-30,73,14:25:00,Sunday,LAROSE AND SCARLETT RD,Diversion,3380.0,368.0,S,8025.0,bus
4,63340,2014-09-02,172,10:37:00,Tuesday,Parliament and Front,Diversion,2430.0,2650.0,w,7830.0,bus
...,...,...,...,...,...,...,...,...,...,...,...,...
479398,22780,2020-07-06,134,12:09:00,Monday,Mammoth Trail/ Washburn way,Diversion,NaN,NaN,S/B,3506.0,bus
479399,23462,2020-07-12,84,14:59:00,Sunday,Sheppard W. STN,Mechanical,NaN,NaN,W/B,3616.0,bus
479400,23875,2020-07-17,52,11:18:00,Friday,Breacrest and The Westway,Mechanical,NaN,NaN,w/b,1230.0,bus
479401,24689,2020-07-26,52,20:54:00,Sunday,6900 AIRPORT RD (INTERNATIONAL CENTR,Investigation,NaN,NaN,E/B,8189.0,bus


In [170]:
df_st.describe()

,Unnamed: 0,Route,Min Delay,Min Gap,Vehicle
count,82702.000000,82702.00000,82702.000000,82702.000000,78017.000000
mean,6403.773488,500.58579,12.991149,12.991149,4405.915826
std,3969.042807,45.97742,31.790575,31.790575,1571.632994
min,0.000000,1.00000,0.000000,0.000000,0.000000
25%,2953.000000,501.00000,5.000000,5.000000,4079.000000
50%,6195.000000,505.00000,6.000000,6.000000,4178.000000
75%,9641.000000,509.00000,11.000000,11.000000,4425.000000
max,15611.000000,999.00000,1400.000000,1400.000000,163242.000000


In [171]:
df_st['vtype'] = 'streetcar'

df_st

,Unnamed: 0,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,vtype
0,0,2014-01-02,505,06:31:00,Thursday,Dundas and Roncesvalles,Late Leaving Garage,4.0,4.0,E/B,4018.0,streetcar
1,1,2014-01-02,504,12:43:00,Thursday,King and Shaw,Utilized Off Route,20.0,20.0,E/B,4128.0,streetcar
2,2,2014-01-02,501,14:01:00,Thursday,Kingston road and Bingham,Held By,13.0,13.0,W/B,4016.0,streetcar
3,3,2014-01-02,504,14:22:00,Thursday,King St. and Roncesvalles Ave.,Investigation,7.0,7.0,W/B,4175.0,streetcar
4,4,2014-01-02,504,16:42:00,Thursday,King and Bathurst,Utilized Off Route,3.0,3.0,E/B,4080.0,streetcar
...,...,...,...,...,...,...,...,...,...,...,...,...
82697,4172,2020-07-26,501,12:58:00,Sunday,QUEEN AND DUFFERIN,Mechanical,10.0,10.0,W/B,4422.0,streetcar
82698,4173,2020-07-27,301,02:09:00,Monday,QUEEN AND YORK,Mechanical,25.0,25.0,E/B,4514.0,streetcar
82699,4174,2020-07-27,306,02:20:00,Monday,Dundas West Station,Emergency Services,19.0,19.0,E/B,3129.0,streetcar
82700,4175,2020-08-03,504,00:56:00,Monday,Dufferin Loop,Mechanical,10.0,10.0,E/B,4464.0,streetcar


In [172]:
# combine bus and streetcar
df_tot = pd.concat([df_bus, df_st])

In [173]:
df_tot.shape

(562105, 12)

In [174]:
df_tot.head(10)

,Unnamed: 0,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,vtype
0,40771,2016-07-24,506,5:40:00,Sunday,Howardpark and Dundas,Diversion,6518.0,6528.0,B/W,7403.0,bus
1,18711,2015-03-19,32,5:00:00,Thursday,Eglinton and Richardson,Diversion,5775.0,5783.0,b/w,NaN,bus
2,15286,2014-02-19,73,18:22:00,Wednesday,ALBION & ISLINGTON,Diversion,3780.0,398.0,S,8044.0,bus
3,87495,2014-11-30,73,14:25:00,Sunday,LAROSE AND SCARLETT RD,Diversion,3380.0,368.0,S,8025.0,bus
4,63340,2014-09-02,172,10:37:00,Tuesday,Parliament and Front,Diversion,2430.0,2650.0,w,7830.0,bus
5,40600,2016-07-23,63,6:00:00,Saturday,Dundas to Queen,Diversion,2422.0,10.0,BW,1055.0,bus
6,27691,2016-05-25,191,6:25:00,Wednesday,Highway 27 at Rexdale Blvd.,Diversion,2030.0,210.0,N/B,7913.0,bus
7,8374,2018-02-07,37,8:00:00,Wednesday,Islington staion -Humberwood loop - Steeles,General Delay,2030.0,50.0,B/W,NaN,bus
8,21617,2014-03-13,108,10:40:00,Thursday,Tuscan Gate and Sheppard Ave West,Diversion,1900.0,208.0,E,1331.0,bus
9,22834,2018-04-23,97,13:26:00,Monday,Yonge and NorthYork center,Diversion,1860.0,1890.0,SB,8183.0,bus


In [175]:
df_tot[df_tot['vtype'] == 'bus'].count() 

Unnamed: 0     479403
Report Date    479403
Route          479403
Time           479403
Day            479403
Location       478609
Incident       478468
Min Delay      455863
Min Gap        460895
Direction      468818
Vehicle        409515
vtype          479403
dtype: int64

In [176]:
df_tot.tail()

,Unnamed: 0,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,vtype
82697,4172,2020-07-26,501,12:58:00,Sunday,QUEEN AND DUFFERIN,Mechanical,10.0,10.0,W/B,4422.0,streetcar
82698,4173,2020-07-27,301,02:09:00,Monday,QUEEN AND YORK,Mechanical,25.0,25.0,E/B,4514.0,streetcar
82699,4174,2020-07-27,306,02:20:00,Monday,Dundas West Station,Emergency Services,19.0,19.0,E/B,3129.0,streetcar
82700,4175,2020-08-03,504,00:56:00,Monday,Dufferin Loop,Mechanical,10.0,10.0,E/B,4464.0,streetcar
82701,4176,2020-08-03,501,23:42:00,Monday,Queen and Connaught (Russell Trailer Track),Investigation,10.0,10.0,W/B,4466.0,streetcar


In [177]:
df_tot = df_tot.drop(columns = 'Unnamed: 0')

In [178]:
df_tot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 562105 entries, 0 to 82701
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Report Date  562105 non-null  object 
 1   Route        562105 non-null  int64  
 2   Time         562105 non-null  object 
 3   Day          562105 non-null  object 
 4   Location     561048 non-null  object 
 5   Incident     561170 non-null  object 
 6   Min Delay    538565 non-null  float64
 7   Min Gap      543597 non-null  float64
 8   Direction    551211 non-null  object 
 9   Vehicle      487532 non-null  float64
 10  vtype        562105 non-null  object 
dtypes: float64(3), int64(1), object(7)
memory usage: 51.5+ MB


## convert report date to datetime format and create year, month and day columns
## convert time to time format to create hour and minute column

In [179]:
df_tot['Report Date'] = pd.to_datetime(df_tot['Report Date'])

In [180]:
df_tot.head(5)

,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,vtype
0,2016-07-24,506,5:40:00,Sunday,Howardpark and Dundas,Diversion,6518.0,6528.0,B/W,7403.0,bus
1,2015-03-19,32,5:00:00,Thursday,Eglinton and Richardson,Diversion,5775.0,5783.0,b/w,NaN,bus
2,2014-02-19,73,18:22:00,Wednesday,ALBION & ISLINGTON,Diversion,3780.0,398.0,S,8044.0,bus
3,2014-11-30,73,14:25:00,Sunday,LAROSE AND SCARLETT RD,Diversion,3380.0,368.0,S,8025.0,bus
4,2014-09-02,172,10:37:00,Tuesday,Parliament and Front,Diversion,2430.0,2650.0,w,7830.0,bus


In [181]:
df_tot = df_tot.reset_index().drop(columns = 'index')

In [182]:
df_tot

,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,vtype
0,2016-07-24,506,5:40:00,Sunday,Howardpark and Dundas,Diversion,6518.0,6528.0,B/W,7403.0,bus
1,2015-03-19,32,5:00:00,Thursday,Eglinton and Richardson,Diversion,5775.0,5783.0,b/w,NaN,bus
2,2014-02-19,73,18:22:00,Wednesday,ALBION & ISLINGTON,Diversion,3780.0,398.0,S,8044.0,bus
3,2014-11-30,73,14:25:00,Sunday,LAROSE AND SCARLETT RD,Diversion,3380.0,368.0,S,8025.0,bus
4,2014-09-02,172,10:37:00,Tuesday,Parliament and Front,Diversion,2430.0,2650.0,w,7830.0,bus
...,...,...,...,...,...,...,...,...,...,...,...
562100,2020-07-26,501,12:58:00,Sunday,QUEEN AND DUFFERIN,Mechanical,10.0,10.0,W/B,4422.0,streetcar
562101,2020-07-27,301,02:09:00,Monday,QUEEN AND YORK,Mechanical,25.0,25.0,E/B,4514.0,streetcar
562102,2020-07-27,306,02:20:00,Monday,Dundas West Station,Emergency Services,19.0,19.0,E/B,3129.0,streetcar
562103,2020-08-03,504,00:56:00,Monday,Dufferin Loop,Mechanical,10.0,10.0,E/B,4464.0,streetcar


In [183]:
df_tot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 562105 entries, 0 to 562104
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Report Date  562105 non-null  datetime64[ns]
 1   Route        562105 non-null  int64         
 2   Time         562105 non-null  object        
 3   Day          562105 non-null  object        
 4   Location     561048 non-null  object        
 5   Incident     561170 non-null  object        
 6   Min Delay    538565 non-null  float64       
 7   Min Gap      543597 non-null  float64       
 8   Direction    551211 non-null  object        
 9   Vehicle      487532 non-null  float64       
 10  vtype        562105 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(6)
memory usage: 47.2+ MB


In [184]:
df_tot['year'] = pd.DatetimeIndex(df_tot['Report Date']).year
df_tot['month'] = pd.DatetimeIndex(df_tot['Report Date']).month
df_tot['day'] = pd.DatetimeIndex(df_tot['Report Date']).day

In [185]:
import datetime as dt

df_tot['hour'] = pd.to_datetime(df_tot['Time']).dt.hour
df_tot['minute'] = pd.to_datetime(df_tot['Time']).dt.minute

In [186]:
df_tot['Location'].str.lower()

0                               howardpark and dundas
1                             eglinton and richardson
2                                  albion & islington
3                              larose and scarlett rd
4                                parliament and front
                             ...                     
562100                             queen and dufferin
562101                                 queen and york
562102                            dundas west station
562103                                  dufferin loop
562104    queen and connaught (russell trailer track)
Name: Location, Length: 562105, dtype: object

In [187]:
df_tot['Location'] = df_tot['Location'].str.lower()
df_tot['Incident'] = df_tot['Incident'].str.lower()
df_tot['Day'] = df_tot['Day'].str.lower()
df_tot['Direction'] = df_tot['Direction'].str.lower()

df_tot

,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,vtype,year,month,day,hour,minute
0,2016-07-24,506,5:40:00,sunday,howardpark and dundas,diversion,6518.0,6528.0,b/w,7403.0,bus,2016,7,24,5,40
1,2015-03-19,32,5:00:00,thursday,eglinton and richardson,diversion,5775.0,5783.0,b/w,NaN,bus,2015,3,19,5,0
2,2014-02-19,73,18:22:00,wednesday,albion & islington,diversion,3780.0,398.0,s,8044.0,bus,2014,2,19,18,22
3,2014-11-30,73,14:25:00,sunday,larose and scarlett rd,diversion,3380.0,368.0,s,8025.0,bus,2014,11,30,14,25
4,2014-09-02,172,10:37:00,tuesday,parliament and front,diversion,2430.0,2650.0,w,7830.0,bus,2014,9,2,10,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562100,2020-07-26,501,12:58:00,sunday,queen and dufferin,mechanical,10.0,10.0,w/b,4422.0,streetcar,2020,7,26,12,58
562101,2020-07-27,301,02:09:00,monday,queen and york,mechanical,25.0,25.0,e/b,4514.0,streetcar,2020,7,27,2,9
562102,2020-07-27,306,02:20:00,monday,dundas west station,emergency services,19.0,19.0,e/b,3129.0,streetcar,2020,7,27,2,20
562103,2020-08-03,504,00:56:00,monday,dufferin loop,mechanical,10.0,10.0,e/b,4464.0,streetcar,2020,8,3,0,56


## Remove null values
## remove rows 

In [188]:
df_tot.isnull().sum()

Report Date        0
Route              0
Time               0
Day                0
Location        1057
Incident         935
Min Delay      23540
Min Gap        18508
Direction      10894
Vehicle        74573
vtype              0
year               0
month              0
day                0
hour               0
minute             0
dtype: int64

In [189]:
df_tot.shape

(562105, 16)

In [190]:
df_tot = df_tot.dropna()

In [191]:
df_tot.isnull().sum()

Report Date    0
Route          0
Time           0
Day            0
Location       0
Incident       0
Min Delay      0
Min Gap        0
Direction      0
Vehicle        0
vtype          0
year           0
month          0
day            0
hour           0
minute         0
dtype: int64

In [192]:
df_tot.shape

(457900, 16)

## Clean location data first prior to retreiving lat and long coordinates

In [193]:
df_tot = df_tot.reset_index()

In [194]:
df_tot['Location'] = df_tot['Location'].str.replace('&', 'and')

In [195]:
df_tot['Location'] = df_tot['Location'].str.replace('stn', 'station')

In [196]:
df_tot['Location'] = df_tot['Location'].str.replace('ave', 'avenue')

In [197]:
df_tot['Location'] = df_tot['Location'].str.replace('str', 'street')

In [198]:
df_tot['Day'] = df_tot['Day'].str.capitalize()

In [199]:
df_tot['Direction'] = df_tot['Direction'].str.replace('/', '')

In [200]:
df_tot = df_tot.drop(columns = 'index')

In [201]:
df_tot['Vehicle'] = df_tot['Vehicle'].astype(int)

In [213]:
df_tot['Location'] = df_tot['Location'].str.replace('/', '')

In [240]:
df_tot['Location'] = df_tot['Location'].str.replace('#', '')
df_tot['Location'] = df_tot['Location'].str.replace('!', '')

In [237]:
df_tot.head(5)

,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,vtype,year,month,day,hour,minute
0,2016-07-24,506,5:40:00,Sunday,howardpark and dundas,diversion,6518.0,6528.0,bw,7403,bus,2016,7,24,5,40
1,2014-02-19,73,18:22:00,Wednesday,albion and islington,diversion,3780.0,398.0,s,8044,bus,2014,2,19,18,22
2,2014-11-30,73,14:25:00,Sunday,larose and scarlett rd,diversion,3380.0,368.0,s,8025,bus,2014,11,30,14,25
3,2014-09-02,172,10:37:00,Tuesday,parliament and front,diversion,2430.0,2650.0,w,7830,bus,2014,9,2,10,37
4,2016-07-23,63,6:00:00,Saturday,dundas to queen,diversion,2422.0,10.0,bw,1055,bus,2016,7,23,6,0


In [397]:
df_tot['Location'].value_counts()[:20]

finch station              10493
kennedy station             9998
warden station              8138
downsview station           7369
eglinton station            7244
kipling station             7151
wilson station              6791
stc                         5390
main station                4827
broadview station           4761
pape station                4507
don mills station           3312
victoria park station       3280
keele station               3202
york mills station          2974
eglinton west station       2675
lawrence station            2652
scarborough town centre     2640
dundas west station         2398
bathurst station            2322
Name: Location, dtype: int64

In [245]:
location = df_tot['Location'].unique().tolist()

len(location)

76270

In [246]:
address= 'howardpark and dundas' + ' Toronto'
loc = geolocator.geocode(address)
if loc != None:
    print(loc.latitude, loc.longitude)
else:
    print(0)

0


## Use geopy first retrieve lat and long coordinates

In [314]:
geolocator = Nominatim(user_agent="my_user_agent")

latlist =[]
longlist =[]

for i in tqdm(location[:10000]):

    address= i + ', Toronto,' + ' Canada'
#    time.sleep(1)
    loc = geolocator.geocode(address)
    if loc != None:
        latlist.append(loc.latitude)
        longlist.append(loc.longitude)
    else:
        latlist.append(0)
        longlist.append(0)        

100%|██████████| 10000/10000 [1:23:24<00:00,  2.00it/s]


In [315]:
geolocator = Nominatim(user_agent="my_user_agent")

latlist2 =[]
longlist2 =[]

for i in tqdm(location[10000:20000]):

    address= i + ', Toronto,' + ' Canada'
#    time.sleep(1)
    loc = geolocator.geocode(address)
    if loc != None:
        latlist2.append(loc.latitude)
        longlist2.append(loc.longitude)
    else:
        latlist2.append(0)
        longlist2.append(0)      

100%|██████████| 10000/10000 [1:24:04<00:00,  1.98it/s] 


In [316]:
geolocator = Nominatim(user_agent="my_user_agent")

latlist3 =[]
longlist3 =[]

for i in tqdm(location[20000:30000]):

    address= i + ', Toronto,' + ' Canada'
#    time.sleep(1)
    loc = geolocator.geocode(address)
    if loc != None:
        latlist3.append(loc.latitude)
        longlist3.append(loc.longitude)
    else:
        latlist3.append(0)
        longlist3.append(0)      

100%|██████████| 10000/10000 [1:23:23<00:00,  2.00it/s]


In [317]:
geolocator = Nominatim(user_agent="my_user_agent")

latlist4 =[]
longlist4 =[]

for i in tqdm(location[30000:40000]):

    address= i + ', Toronto,' + ' Canada'
#    time.sleep(1)
    loc = geolocator.geocode(address)
    if loc != None:
        latlist4.append(loc.latitude)
        longlist4.append(loc.longitude)
    else:
        latlist4.append(0)
        longlist4.append(0)   

100%|██████████| 10000/10000 [1:23:24<00:00,  2.00it/s]


In [318]:
geolocator = Nominatim(user_agent="my_user_agent")

latlist5 =[]
longlist5 =[]

for i in tqdm(location[40000:50000]):

    address= i + ', Toronto,' + ' Canada'
#    time.sleep(1)
    loc = geolocator.geocode(address)
    if loc != None:
        latlist5.append(loc.latitude)
        longlist5.append(loc.longitude)
    else:
        latlist5.append(0)
        longlist5.append(0)   

100%|██████████| 10000/10000 [1:23:23<00:00,  2.00it/s]


In [319]:
geolocator = Nominatim(user_agent="my_user_agent")

latlist6 =[]
longlist6 =[]

for i in tqdm(location[50000:60000]):

    address= i + ', Toronto,' + ' Canada'
#    time.sleep(1)
    loc = geolocator.geocode(address)
    if loc != None:
        latlist6.append(loc.latitude)
        longlist6.append(loc.longitude)
    else:
        latlist6.append(0)
        longlist6.append(0)   

100%|██████████| 10000/10000 [1:23:23<00:00,  2.00it/s]


In [320]:
geolocator = Nominatim(user_agent="my_user_agent")

latlist7 =[]
longlist7 =[]

for i in tqdm(location[60000:70000]):

    address= i + ', Toronto,' + ' Canada'
#    time.sleep(1)
    loc = geolocator.geocode(address)
    if loc != None:
        latlist7.append(loc.latitude)
        longlist7.append(loc.longitude)
    else:
        latlist7.append(0)
        longlist7.append(0)   

100%|██████████| 10000/10000 [1:23:25<00:00,  2.00it/s]


In [321]:
geolocator = Nominatim(user_agent="my_user_agent")

latlist8 =[]
longlist8 =[]

for i in tqdm(location[70000:]):

    address= i + ', Toronto,' + ' Canada'
#    time.sleep(1)
    loc = geolocator.geocode(address)
    if loc != None:
        latlist8.append(loc.latitude)
        longlist8.append(loc.longitude)
    else:
        latlist8.append(0)
        longlist8.append(0)   

100%|██████████| 6270/6270 [52:16<00:00,  2.00it/s] 


In [322]:
latlist_total = latlist + latlist2 + latlist3 + latlist4 + latlist5 + latlist6 + latlist7 + latlist8
longlist_total = longlist + longlist2 + longlist3 + longlist4 + longlist5 + longlist6 + longlist7 + longlist8

print(len(latlist_total))
print(len(longlist_total))

76270
76270


In [323]:
d = {'location': location, 'lat': latlist_total, 'long': longlist_total}

In [324]:
df_location = pd.DataFrame(data = d)

df_location
print(len(df_location[df_location['lat'] != 0.000000]))
print(len(df_location[df_location['lat'] == 0.000000]))

#lat: 42
#long: 58

19094
57176


In [345]:
df_location[:40000][df_location['lat'] == 0.000000]

<ipython-input-345-e8620d7f1c99>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_location[:40000][df_location['lat'] == 0.000000]


,location,lat,long
0,howardpark and dundas,0.0,0.0
2,larose and scarlett rd,0.0,0.0
5,highway 27 at rexdale blvd.,0.0,0.0
7,yonge and northyork center,0.0,0.0
8,bathurst and feets,0.0,0.0
...,...,...,...
39992,eglinton and thermos,0.0,0.0
39993,she4ppard and bayview,0.0,0.0
39996,bathurst and antibies,0.0,0.0
39998,wavenuerlykingston rd,0.0,0.0


In [325]:
df_location.to_csv('location_coor.csv')

In [328]:
df_location

,location,lat,long
0,howardpark and dundas,0.000000,0.000000
1,albion and islington,43.736643,-79.564858
2,larose and scarlett rd,0.000000,0.000000
3,parliament and front,43.651438,-79.362966
4,dundas to queen,43.652732,-79.398948
...,...,...,...
76265,bradview and langley avenue,0.000000,0.000000
76266,college shaw,43.670463,-79.383867
76267,cherry and eastern avenue,0.000000,0.000000
76268,college st west rushholme,0.000000,0.000000


In [329]:
latlist_g = df_location['lat'].tolist()
longlist_g = df_location['long'].tolist()

In [351]:
len(location)

76270

In [352]:
len(latlist_g)

76270

## Retrieve remaining missing lat and long coordinates using googlemaps API

In [410]:
latlist_g2 = []
longlist_g2 = []
location_g2 = []


for i,j,k in tqdm(zip(latlist_g, longlist_g, location)):
    
    try: 
        if i == 0:
            
            address_c = k + ', Toronto,' + ' Canada'

            gmaps = googlemaps.Client(key='')

            try: 
                geocode_result = gmaps.geocode(address_c)
                if geocode_result != []:
                    try:
                        lat = geocode_result[0]['geometry']['location']['lat']
                        #print(lat)
                        long = geocode_result[0]['geometry']['location']['lng']
                        #print(long)
                        latlist_g2.append(lat)
                        longlist_g2.append(long)
                        location_g2.append(k)
                    
                    except:
                        latlist_g2.append(i)
                        longlist_g2.append(j)
                        location_g2.append(k)   

                else: 
                    latlist_g2.append(i)
                    longlist_g2.append(j)
                    location_g2.append(k)      

            except:
                latlist_g2.append(i)
                longlist_g2.append(j)
                location_g2.append(k) 

        else:
            latlist_g2.append(i)
            longlist_g2.append(j)
            location_g2.append(k)

    except:
        latlist_g2.append(i)
        longlist_g2.append(j)
        location_g2.append(k) 
            
            

# Look up an address with reverse geocoding
#reverse_geocode_result = gmaps.reverse_geocode((40.714224, -73.961452))

# Request directions via public transit

76270it [10:18:49,  2.05it/s]


In [411]:
print(len(latlist_g2))
print(len(longlist_g2))

76270
76270


In [358]:
len(latlist_g)

76270

In [364]:
latlist_g

[0.0,
 43.7366425,
 0.0,
 43.6514382,
 43.6527322,
 0.0,
 43.752992,
 0.0,
 0.0,
 43.6871776,
 0.0,
 0.0,
 43.6481827,
 0.0,
 0.0,
 43.6602019,
 43.6659145,
 0.0,
 0.0,
 43.635240350000004,
 43.7061229,
 43.7074907,
 0.0,
 0.0,
 43.6485131,
 43.69009395,
 0.0,
 43.7230093,
 43.7894115,
 0.0,
 0.0,
 43.7053539,
 43.731752,
 43.731752,
 43.6482754,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 43.616890749999996,
 0.0,
 0.0,
 43.6516516,
 0.0,
 43.6853852,
 0.0,
 0.0,
 43.7341884,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 43.788073,
 0.0,
 43.6561162,
 0.0,
 0.0,
 43.649826,
 0.0,
 0.0,
 43.63794985,
 0.0,
 43.64984125,
 0.0,
 43.7176064,
 43.676735050000005,
 0.0,
 0.0,
 43.6577734,
 43.7901172,
 0.0,
 43.7322688,
 0.0,
 43.6548771,
 0.0,
 43.7749246,
 43.7594449,
 0.0,
 43.7812974,
 43.647942,
 0.0,
 0.0,
 0.0,
 0.0,
 43.7864156,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 43.70275905,
 43.7010547,
 43.6633629,
 0.0,
 0.0,
 43.6898368,
 43.6898368,
 43.67996425,
 43.67996425,
 43.66336049999999

In [359]:
len(longlist_g)

76270

In [412]:
len(latlist_g2)

76270

In [413]:
len(longlist_g2)

76270

In [362]:
len(location)

76270

In [414]:
d = {'location': location, 'lat': latlist_g2, 'long': longlist_g2}

In [418]:
df_location = pd.DataFrame(data = d)

df_location.set_index('location')

df_location = df_location.rename(columns = {'location': 'Location'})
df_location.set_index('Location')

,lat,long
Location,,
howardpark and dundas,43.652162,-79.448173
albion and islington,43.736643,-79.564858
larose and scarlett rd,43.686860,-79.513081
parliament and front,43.651438,-79.362966
dundas to queen,43.652732,-79.398948
...,...,...
bradview and langley avenue,43.667812,-79.353345
college shaw,43.670463,-79.383867
cherry and eastern avenue,43.653834,-79.358610


In [416]:
df_location.loc[39999]

Location    queenquay and sherbourne
lat                          43.6453
long                        -79.3653
Name: 39999, dtype: object

In [394]:
latlist_g2[39999]

43.7605272

## random test case for coordinates

In [391]:
df_location[df_location['Location'] == 'sloane and sweeney']

,Location,lat,long
2466,sloane and sweeney,43.749217,-79.510386


In [392]:
df_location[df_location['Location'] == 'exhibition place']

,Location,lat,long
2468,exhibition place,43.672473,-79.467535


## Combine location data to bus and streetcar data using location common column

In [419]:
df_tot_v1 = pd.merge(df_tot, df_location)

In [420]:
df_tot_v1

,Report Date,Route,Time,Day,Location,Incident,Min Delay,Min Gap,Direction,Vehicle,vtype,year,month,day,hour,minute,lat,long
0,2016-07-24,506,5:40:00,Sunday,howardpark and dundas,diversion,6518.0,6528.0,bw,7403,bus,2016,7,24,5,40,43.652162,-79.448173
1,2014-02-19,73,18:22:00,Wednesday,albion and islington,diversion,3780.0,398.0,s,8044,bus,2014,2,19,18,22,43.736643,-79.564858
2,2016-02-01,73,7:12:00,Monday,albion and islington,diversion,53.0,68.0,e,8049,bus,2016,2,1,7,12,43.736643,-79.564858
3,2015-08-17,73,22:29:00,Monday,albion and islington,diversion,20.0,41.0,e,7956,bus,2015,8,17,22,29,43.736643,-79.564858
4,2015-10-18,37,11:28:00,Sunday,albion and islington,mechanical,20.0,40.0,sb,7901,bus,2015,10,18,11,28,43.736643,-79.564858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457895,2020-07-19,504,12:05:00,Sunday,bradview and langley avenue,held by,110.0,110.0,wb,4580,streetcar,2020,7,19,12,5,43.667812,-79.353345
457896,2020-07-19,506,12:59:00,Sunday,college shaw,investigation,8.0,8.0,eb,8517,streetcar,2020,7,19,12,59,43.670463,-79.383867
457897,2020-07-19,504,13:40:00,Sunday,cherry and eastern avenue,overhead - pantograph,28.0,28.0,eb,4532,streetcar,2020,7,19,13,40,43.653834,-79.358610
457898,2020-07-19,506,14:23:00,Sunday,college st west rushholme,emergency services,30.0,30.0,eb,8057,streetcar,2020,7,19,14,23,43.655219,-79.413860


In [421]:
df_tot_v1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 457900 entries, 0 to 457899
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Report Date  457900 non-null  datetime64[ns]
 1   Route        457900 non-null  int64         
 2   Time         457900 non-null  object        
 3   Day          457900 non-null  object        
 4   Location     457900 non-null  object        
 5   Incident     457900 non-null  object        
 6   Min Delay    457900 non-null  float64       
 7   Min Gap      457900 non-null  float64       
 8   Direction    457900 non-null  object        
 9   Vehicle      457900 non-null  int32         
 10  vtype        457900 non-null  object        
 11  year         457900 non-null  int64         
 12  month        457900 non-null  int64         
 13  day          457900 non-null  int64         
 14  hour         457900 non-null  int64         
 15  minute       457900 non-null  int6

In [422]:
df_tot_v1.to_csv('BusandStreetcarDelayData_v2.csv')